# Bokeh Demo 2

*Create a version with widgets, adapted from here :*

http://bokeh.pydata.org/en/latest/docs/gallery/slider.html

In [1]:
import numpy as np
from bokeh.plotting import *
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource, CustomJS, Slider

#output the plots to this notebook
output_notebook()

# If you are using the Javascript callback, you can uncomment this line below to save the plot as 
#    a stand alone html file that you could use in your website.
# output_file("linked_brushing_slider.html", title='Bokeh demo 2')

## 1. Prepare some data.

In [2]:
N = 300
x = np.linspace(0, 4*np.pi, N)
y0 = np.sin(x)
y1 = np.cos(x)

## 2. Define the "ColumnDataSource" and the plots.

*A ColumnDataSource will hold a python dictionary (or a panda dataframe) containing your data and can be accessed by Bokeh.*

In [3]:
# create a column data source for the plots to share
source = ColumnDataSource(data=dict(x=x, y0=y0, y1=y1))

TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select,lasso_select"

# create a new plot and add a renderer
left = figure(tools=TOOLS, width=350, height=350, title=None)
left.line('x', 'y0', source=source, color='red', line_width=2, line_alpha=0.4)
left.circle('x', 'y0', source=source, color='red')
left.xaxis.axis_label = 'x'
left.yaxis.axis_label = 'sin(x)'

# create another new plot and add a renderer
right = figure(tools=TOOLS, width=350, height=350, title=None)
right.line('x', 'y1', source=source, color='blue', line_width=2, line_alpha=0.4)
right.circle('x', 'y1', source=source, color='blue')
right.xaxis.axis_label = 'x'
right.yaxis.axis_label = 'cos(x)'

plots = [left, right]


## 3. Define the layout.

*I will write this as a function so that we can it reuse it.  (See below.)*


In [4]:
def defineLayout(plots, sliders):
    # put the subplots in a gridplot
    p = gridplot([plots])

    #now add the sliders
    layout = row(
        p,
        column(*sliders), #the asterisk unpacks the list of sliders into individual values
    )
    
    return layout

## 4. Define some sliders that will control the data that is plotted.

*This bit below will simply define the sliders that we want to use.  Further down I will connect a ("callback") function that manipulates the data when a slider changes.*

*There are many different kinds of widgets that can be used in Bokeh.  [Examples can be found here.](https://docs.bokeh.org/en/latest/docs/user_guide/interaction/widgets.html)*




In [5]:
# define the sliders
amp_slider = Slider(start=0.1, end=10, value=1, step=.1, title="Amplitude")
freq_slider = Slider(start=0.1, end=10, value=1, step=.1, title="Frequency")
phase_slider = Slider(start=0, end=6.4, value=0, step=.1,title="Phase")
offset_slider = Slider(start=-5, end=5, value=0, step=.1,title="Offset")

sliders_base = [amp_slider, freq_slider, phase_slider, offset_slider]

## 5. Define a "callback" to control the sliders and link it in.

*A callback is a generic term for a function that is called after some event happens.  Here the function will be called after a slider's value is changed.*

*I will create two different versions, one in Python and one in Javascript.  Only one should be used at a time.  (You may have to restart the notebook kernel in order to switch the callback method.)  I will wrap each of these in separate functions so that it's easier to swap one in for the other.*


### 5a. The Javascript approach:

*In this example below, I am writing the callback function in Javascript.  This will allow us to show the result within a jupyter (and colab) notebook and also to save the resulting plot as a standalone .html file that could be used on your website.  (But, you need to learn a little Javascript.)*

In [6]:
def defineJSSliders(source, sliders_base):

    amp_slider, freq_slider, phase_slider, offset_slider = sliders_base
    
    # write the javascript slider callback
    sliderCallback = CustomJS(args=dict(source=source, amp=amp_slider, freq=freq_slider, phase=phase_slider, 
                                   offset=offset_slider), code="""
        // Get the current slider values
        var A = amp.value;
        var k = freq.value;
        var phi = phase.value;
        var B = offset.value;

        // Generate the new data
        var x = source.data['x'];
        var y0 = source.data['y0'];
        var y1 = source.data['y1'];
        for (var i = 0; i < x.length; i++) {
            y0[i] = B + A*Math.sin(k*x[i]+phi);
            y1[i] = B + A*Math.cos(k*x[i]+phi);
        }

        // Redefine the data in the ColumnDataSource
        source.change.emit();
    """)

    # connect the callback to the sliders
    amp_slider.js_on_change('value', sliderCallback)
    freq_slider.js_on_change('value', sliderCallback)
    phase_slider.js_on_change('value', sliderCallback)
    offset_slider.js_on_change('value', sliderCallback)
    
    return amp_slider, freq_slider, phase_slider, offset_slider

*Connect the sliders, define the layout and show the plots.*



In [7]:
sliders = defineJSSliders(source, sliders_base)
layout = defineLayout(plots, sliders)
show(layout)

### 5b. The Python approach:

*In this example below, I am writing the callback function in Python.  This approach has the benefit of already being in a language you know (Python), but you cannot use Python callbacks to create an .html file.  Only Javascript callbacks can be used to create an interactive plot for your website.  (Also Python callbacks will not work in colab.)*  

In [ ]:
def definePySliders(source, sliders_base):
    
    amp_slider, freq_slider, phase_slider, offset_slider = sliders_base

    #write the python slider callback
    def sliderCallback(attrname, old, new):

        # Get the current slider values
        A = amp_slider.value
        k = freq_slider.value
        phi = phase_slider.value
        B = offset_slider.value

        # Generate the new data
        x = source.data['x']
        y0 = A*np.sin(k*x + phi) + B
        y1 = A*np.cos(k*x + phi) + B

        # Redefine the data in the ColumnDataSource
        source.data = dict(x=x, y0=y0, y1=y1)
        
    # connect the callback to the sliders
    amp_slider.on_change('value', sliderCallback)
    freq_slider.on_change('value', sliderCallback)
    phase_slider.on_change('value', sliderCallback)
    offset_slider.on_change('value', sliderCallback)
    
    return amp_slider, freq_slider, phase_slider, offset_slider


*Connect the sliders, define the layout and show the plots.*

In [ ]:
# This function here enables you to run a bokeh script with a python callback in the notebook. 
# (Otherwise you could write the code as a .py file and run it through a bokeh server.)
def bkapp(doc):
    doc.add_root(layout)

sliders = defineJSSliders(source, sliders_base)
layout = defineLayout(plots, sliders)
show(bkapp)